In [ ]:
from keras.layers import Dense,Dropout
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd 
import os

import efficientnet.keras as efn
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import densenet


In [ ]:
from keras.optimizers import Adam
optimizer = Adam(lr = 0.0003 , decay = 0.000001)

In [ ]:
from keras.callbacks import EarlyStopping , ModelCheckpoint
es = EarlyStopping(patience=3)
mc = ModelCheckpoint("tongue_b6_acc5.h5",save_best_only = True)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.2,0.8],
    validation_split=0.2,
)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_dir = "./Capture/smoke_con/train/"
valid_dir = "./Capture/smoke_con/validation/"
test_dir = "./Capture/smoke_con/test/"

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300,300),
    batch_size=25,
    class_mode='binary')
valid_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(300,300),
    batch_size=25,
    class_mode='binary')

In [ ]:
model1 = Sequential()
model1.add(efn.EfficientNetB1(weights='imagenet', include_top = False, pooling='avg'))
model1.add(Dense(1,activation="sigmoid"))
model1.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=["acc"])

hist1 = model1.fit(
    train_generator,
    steps_per_epoch =5,
    epochs = 100,
    validation_data = valid_generator,
    validation_steps=5,
    callbacks = [es,mc]
)

In [ ]:
model2 = Sequential()
model2.add(VGG16(weights='imagenet', include_top = False, pooling='avg'))
model2.add(Dense(1,activation="sigmoid"))
model2.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=["acc"])

hist2 = model2.fit_generator(
    train_generator,
    epochs = 100,
    validation_data = valid_generator
)

In [ ]:
model3 = Sequential()
model3.add(ResNet50(weights='imagenet', include_top = False, pooling='avg'))
model3.add(Dense(1,activation="sigmoid"))
model3.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=["acc"])

hist3 = model3.fit_generator(
    train_generator,
    epochs = 100,
    validation_data = valid_generator
)

In [ ]:
model4 = Sequential()
model4.add(densenet.DenseNet121(weights='imagenet', include_top = False, pooling='avg'))
model4.add(Dense(1,activation="sigmoid"))
model4.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=["acc"])

hist4 = model4.fit_generator(
    train_generator,
    epochs = 100,
    validation_data = valid_generator
)

In [ ]:
eva_train1 = model1.evaluate_generator(train_generator,workers = 4,verbose=1)
eva_train2 = model2.evaluate_generator(train_generator,workers = 4,verbose=1)
eva_train3 = model3.evaluate_generator(train_generator,workers = 4,verbose=1)
eva_train4 = model4.evaluate_generator(train_generator,workers = 4,verbose=1)


print('EfficientNetB1: '+str(eva_train1[1]))
print('VGG16: '+str(eva_train2[1]))
print('ResNet50: '+str(eva_train3[1]))
print('DenseNet121: '+str(eva_train4[1]))

In [ ]:
import matplotlib.pyplot as plt

acc1 = hist1.history['acc']
acc2 = hist2.history['acc']
acc3 = hist3.history['acc']
acc4 = hist4.history['acc']

plt.plot(acc1, label='EfficientNetB6')
plt.plot(acc2, label='VGG16')
plt.plot(acc3, label='ResNet50')
plt.plot(acc4, label='DenseNet121')

plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
loss1 = hist1.history['loss']
loss2 = hist2.history['loss']
loss3 = hist3.history['loss']
loss4 = hist4.history['loss']

plt.plot(loss1, label='EfficientNetB6')
plt.plot(loss2, label='VGG16')

plt.plot(loss3, label='ResNet50')
plt.plot(loss4, label='DenseNet121')

plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                          target_size=(300, 300),
                                          batch_size=25,
                                          class_mode='binary')

In [ ]:
evaluate1 = model1.evaluate_generator(test_generator,workers = 4,verbose=1)
evaluate2 = model2.evaluate_generator(test_generator,workers = 4,verbose=1)
evaluate3 = model3.evaluate_generator(test_generator,workers = 4,verbose=1)
evaluate4 = model4.evaluate_generator(test_generator,workers = 4,verbose=1)


print('EfficientNetB1: '+str(evaluate1[1]))
print('VGG16: '+str(evaluate2[1]))
print('ResNet50: '+str(evaluate3[1]))
print('DenseNet121: '+str(evaluate4[1]))